In [1]:
!pip install -q -U transformers peft accelerate optimum  #lets install transformer peft accelerate and optium

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 29.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.6/85.6 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.2/251.2 kB 26.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.6/380.6 kB 41.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 28.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 85.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 77.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 50.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 68.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

we have to use autogptq for now from source directly

In [2]:
!pip install auto-gptq --extra-index-url https://huggingface.github.io/autogptq-index/whl/cu117/

Looking in indexes: https://pypi.org/simple, https://huggingface.github.io/autogptq-index/whl/cu117/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 11.8 MB/s eta 0:00:00


In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer, GPTQConfig
import torch

model_id = "facebook/opt-125m"#very small model for the sake of checking for LLM project right now

quantization_config = GPTQConfig( #this the hugging face library for gptq quantization
     bits=4,#here we did a 4 bit quatization
     group_size=128,
     dataset="c4",#standard dataset to tune the quatizied weights
     desc_act=False,
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
quant_model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=quantization_config, device_map='auto')#here model is getting quantized can take 5 min

Repo card metadata block was not found. Setting CardData to empty.


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Quantizing model.decoder.layers blocks :   0%|          | 0/12 [00:00<?, ?it/s]

Quantizing layers inside the block:   0%|          | 0/6 [00:00<?, ?it/s]

...

Quantizing layers inside the block:   0%|          | 0/6 [00:00<?, ?it/s]

Quantizing layers inside the block:   0%|          | 0/6 [00:00<?, ?it/s]

Quantizing layers inside the block:   0%|          | 0/6 [00:00<?, ?it/s]

Quantizing layers inside the block:   0%|          | 0/6 [00:00<?, ?it/s]

Quantizing layers inside the block:   0%|          | 0/6 [00:00<?, ?it/s]

In [4]:
quant_model.model.decoder.layers[0].self_attn.q_proj.__dict__#just to check that if the weights are quantized this makes sure weight is quantized but not make sure the finetuning is good
#we can check here if the weights are in 32bit float as we did 4 bit quantization

{'training': True,
 '_parameters': OrderedDict(),
 '_buffers': OrderedDict([('qweight',
               tensor([[ 1711760090, -1248295259, -2025411892,  ..., -1486452502,
                         2019142072, -1735820810],
                       [-2000132747,  -578262345,  1484081337,  ..., -1230600537,
                        -2019252040, -2023311003],
                       [ -710293851, -1153090188,  1431922298,  ..., -1768449094,
                         2042194587, -2004125258],
                       ...,
                       [-1183500136, -1494510422, -1772782904,  ..., -1518753378,
                         -411710600,  -392845654],
                       [-1990626701,  1469278281,  1469864108,  ...,  1740208533,
                        -1732560507, -1738077576],
                       [ 2015914598,  2040232821,  2005572185,  ..., -1463179655,
                        -1450400136, -2024523156]], device='cuda:0', dtype=torch.int32)),
              ('qzeros',
               tensor(

In [14]:
tokenizer = AutoTokenizer.from_pretrained(model_id)#same tokenizer is used for quantized model

text =  "I am a LLM "
inputs = tokenizer(text, return_tensors="pt").to(0)

out = quant_model.generate(**inputs)
print(tokenizer.decode(out[0], skip_special_tokens=True))#output is kind off meh just coz of very small size of model m=same approch can work for lambda models as well

I am a LLM   I am a student of the University of California at Berkeley.


In [16]:
import pandas as pd
df=pd.read_csv("Tech.csv") #this the primiliary discord dataset

In [64]:
list1=[]
num=0
for i in df["Content"]:
  list1.append(str(i))
  num+=1
  if(num==1024):
    break


In [65]:
len(list1)

1024

In [66]:
from transformers import AutoModelForCausalLM, GPTQConfig, AutoTokenizer#trying to train on custom dataset

model_id = "facebook/opt-125m"

quantization_config = GPTQConfig(
    bits=4,
    group_size=128,
    desc_act=False,
    dataset=list1#dataset should be strictly list of strings
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
quant_model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=quantization_config, torch_dtype=torch.float16, device_map="auto")

Quantizing model.decoder.layers blocks :   0%|          | 0/12 [00:00<?, ?it/s]

Quantizing layers inside the block:   0%|          | 0/6 [00:00<?, ?it/s]

Quantizing layers inside the block:   0%|          | 0/6 [00:00<?, ?it/s]

Quantizing layers inside the block:   0%|          | 0/6 [00:00<?, ?it/s]

Quantizing layers inside the block:   0%|          | 0/6 [00:00<?, ?it/s]

...

Quantizing layers inside the block:   0%|          | 0/6 [00:00<?, ?it/s]

Quantizing layers inside the block:   0%|          | 0/6 [00:00<?, ?it/s]

Quantizing layers inside the block:   0%|          | 0/6 [00:00<?, ?it/s]

Quantizing layers inside the block:   0%|          | 0/6 [00:00<?, ?it/s]

Quantizing layers inside the block:   0%|          | 0/6 [00:00<?, ?it/s]

As you can see from the generation below, the performance seems to be slightly worse than the model quantized using the `c4` dataset.

In [77]:
text = "hello,i am justin beiber and i love "
inputs = tokenizer(text, return_tensors="pt").to(0)

out = quant_model.generate(**inputs)
print(tokenizer.decode(out[0], skip_special_tokens=True))#output is repeating again coz of small size + set max no. of token

hello,i am justin beiber and i love  you
I'm justin Be


## Share quantized models on 🤗 Hub

After quantizing the model, it can be used out-of-the-box for inference or you can push the quantized weights on the 🤗 Hub to share your quantized model with the community

In [69]:
from huggingface_hub import notebook_login

notebook_login()

In [70]:
quant_model.push_to_hub("rohit_opt-125m-gptq-4bit")
tokenizer.push_to_hub("rohit_opt-125m-gptq-4bit")

pytorch_model.bin:   0%|          | 0.00/125M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/rjindal/rohit_opt-125m-gptq-4bit/commit/382a4c3b92c020dbec33ee8f88e4c6a6f34fc821', commit_message='Upload tokenizer', commit_description='', oid='382a4c3b92c020dbec33ee8f88e4c6a6f34fc821', pr_url=None, pr_revision=None, pr_num=None)

In [78]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_id = "rjindal/rohit_opt-125m-gptq-4bit"
model = AutoModelForCausalLM.from_pretrained(model_id, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(model_id)

## Exploring further